In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.model_selection import StratifiedKFold
import time

data_path = os.path.join('.','tests','test_output')

In [2]:
y_true = []

knn_y = []
svm_y = []
dt_y = []

for i in range(8):
    pca = PCA(n_components = 100)
    X = np.load(os.path.join(data_path,'x_split8_{}.npy'.format(i)),mmap_mode='r')
    Y = np.load(os.path.join(data_path,'y_split8_{}.npy'.format(i)),mmap_mode='r')
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, stratify = Y)
    
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    knn = KNeighborsClassifier(n_neighbors=1)
    svm = SVC(C=1000,kernel = 'linear')
    dt = tree.DecisionTreeClassifier()
    
    knn.fit(X_train,y_train)
    svm.fit(X_train,y_train)
    dt.fit(X_train,y_train)
    
    y_true.append(y_test)
    
    knn_y.append(knn.predict(X_test))
    svm_y.append(svm.predict(X_test))
    dt_y.append(dt.predict(X_test))
    

In [3]:
from mlxtend.evaluate import cochrans_q

for i in range(8):
    q, p_value = cochrans_q(y_true[i], 
                        knn_y[i], 
                        svm_y[i], 
                        dt_y[i])
    print("q:{},p:{}".format(q, p_value))

q:28.04206500956023,p:8.142222247858422e-07
q:19.319311663479922,p:6.380647831761894e-05
q:54.06026365348399,p:1.8237399116478303e-12
q:18.666666666666668,p:8.8426988659883e-05
q:37.870370370370374,p:5.977967646550633e-09
q:27.749532710280373,p:9.42465658266892e-07
q:7.833655705996131,p:0.01990413354488903
q:39.02048417132216,p:3.3636401021166876e-09


C:\Users\Msam\.conda\envs\machine_learning\lib\site-packages\mlxtend\evaluate\cochrans_q.py:75: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  mod in y_model_predictions)).astype(int)


In [4]:
for i in range(8):
    knn_sc = np.mean(y_true[i]==knn_y[i])
    print("knn score:{}".format(knn_sc))
    
    svm_sc = np.mean(y_true[i]==svm_y[i])
    print("svm score:{}".format(svm_sc))
    
    dt_sc = np.mean(y_true[i]==dt_y[i])
    print("dt score:{}".format(dt_sc))

knn score:0.5171717171717172
svm score:0.5050505050505051
dt score:0.42525252525252527
knn score:0.5
svm score:0.5383838383838384
dt score:0.45555555555555555
knn score:0.5171717171717172
svm score:0.5444444444444444
dt score:0.4121212121212121
knn score:0.5070707070707071
svm score:0.5373737373737374
dt score:0.45656565656565656
knn score:0.4858585858585859
svm score:0.5262626262626262
dt score:0.4101010101010101
knn score:0.503030303030303
svm score:0.509090909090909
dt score:0.41919191919191917
knn score:0.5141414141414141
svm score:0.5141414141414141
dt score:0.4686868686868687
knn score:0.5151515151515151
svm score:0.5595959595959596
dt score:0.4414141414141414
